# Imports

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import seaborn as sns
import google.colab
import numpy as np
from sklearn.feature_selection import mutual_info_regression
import scipy.stats as stats
from sklearn.feature_selection import f_classif

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
health_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/project/Wearable/personal_health_data.csv")

In [5]:
health_df.head(2000)

,User_ID,Age,Gender,Weight,Height,Medical_Conditions,Medication,Smoker,Alcohol_Consumption,Timestamp,...,ECG,Calories_Intake,Water_Intake,Stress_Level,Mood,Skin_Temperature,Body_Fat_Percentage,Muscle_Mass,Health_Score,Anomaly_Flag
0,U_00001,61,Other,88.677623,178.524231,Diabetes,No,No,Moderate,2021-01-01 00:00:00,...,Abnormal,2444.985808,3.105556,Moderate,Neutral,32.752993,17.432985,77.599448,25.992071,1
1,U_00002,24,Male,86.742084,200.346358,NaN,Yes,No,Moderate,2021-01-01 01:00:00,...,Normal,2199.777020,2.441118,High,Anxious,35.010044,23.957792,71.718936,64.308450,0
2,U_00003,81,Other,61.656054,195.971356,Hypertension,Yes,No,Moderate,2021-01-01 02:00:00,...,Abnormal,2182.755640,1.240341,Low,Sad,36.797523,22.995533,59.874936,78.003546,1
3,U_00004,70,Other,141.513794,164.178216,Hypertension,No,No,Heavy,2021-01-01 03:00:00,...,Normal,1451.218888,1.682383,Moderate,Happy,32.614592,13.410659,67.528098,16.454106,1
4,U_00005,30,Other,80.552872,146.920675,Diabetes,Yes,Yes,Heavy,2021-01-01 04:00:00,...,Normal,1583.562773,1.461862,High,Happy,33.602319,24.920020,54.741487,42.528202,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,U_01996,35,Male,34.696250,146.773361,Diabetes,No,No,Moderate,2021-03-25 03:00:00,...,Normal,1769.373503,3.285633,Low,Sad,33.129438,22.679256,34.815523,26.969795,1
1996,U_01997,77,Other,34.218775,170.488833,Hypertension,No,Yes,Moderate,2021-03-25 04:00:00,...,Normal,2955.734481,2.108255,Low,Sad,32.636778,33.069656,25.202274,22.709383,1
1997,U_01998,64,Female,40.961148,189.482189,Hypertension,Yes,Yes,Heavy,2021-03-25 05:00:00,...,Normal,1575.103242,2.458705,Low,Sad,33.998388,30.142384,76.042177,34.551272,0
1998,U_01999,15,Other,72.722820,179.933871,NaN,Yes,No,Moderate,2021-03-25 06:00:00,...,Normal,2884.127401,2.548131,Moderate,Anxious,32.276296,30.737913,21.787574,87.943254,0


In [39]:
pizza = health_df.copy()

In [40]:
pizza.groupby('Mood').size()

,0
Mood,
Anxious,2542
Happy,2448
Neutral,2495
Sad,2515


In [41]:
print(len(pizza))

10000


In [44]:
# prompt: find me rows wgere somewhere theres 'other' value or value that supposd to be a number but it isnt

import pandas as pd

# Assuming health_df is already loaded as in your previous code
# ... (your existing code to load the dataframe) ...

def find_bad_rows(df):
    """Finds rows with 'other' values or non-numeric values where numbers are expected."""
    bad_rows_indices = []

    # Iterate through each column
    for col in df.columns:
      # Handle cases where columns might be numeric
        if pd.api.types.is_numeric_dtype(df[col]):
          # Attempt to convert to numeric, catching errors for non-numeric values
            try:
                pd.to_numeric(df[col])
            except (ValueError, TypeError):
                bad_rows_indices.extend(df[pd.to_numeric(df[col], errors='coerce').isnull()].index)

        elif 'other' in df[col].astype(str).str.lower().values:
            bad_rows_indices.extend(df[df[col].astype(str).str.lower().str.contains('other')].index)

    # Return only unique bad row indices
    return df.iloc[list(set(bad_rows_indices))]

bad_rows = find_bad_rows(health_df)
bad_rows


,User_ID,Age,Gender,Weight,Height,Medical_Conditions,Medication,Smoker,Alcohol_Consumption,Timestamp,...,ECG,Calories_Intake,Water_Intake,Stress_Level,Mood,Skin_Temperature,Body_Fat_Percentage,Muscle_Mass,Health_Score,Anomaly_Flag
0,U_00001,61,Other,88.677623,178.524231,Diabetes,No,No,Moderate,2021-01-01 00:00:00,...,Abnormal,2444.985808,3.105556,Moderate,Neutral,32.752993,17.432985,77.599448,25.992071,1
2,U_00003,81,Other,61.656054,195.971356,Hypertension,Yes,No,Moderate,2021-01-01 02:00:00,...,Abnormal,2182.755640,1.240341,Low,Sad,36.797523,22.995533,59.874936,78.003546,1
3,U_00004,70,Other,141.513794,164.178216,Hypertension,No,No,Heavy,2021-01-01 03:00:00,...,Normal,1451.218888,1.682383,Moderate,Happy,32.614592,13.410659,67.528098,16.454106,1
4,U_00005,30,Other,80.552872,146.920675,Diabetes,Yes,Yes,Heavy,2021-01-01 04:00:00,...,Normal,1583.562773,1.461862,High,Happy,33.602319,24.920020,54.741487,42.528202,0
5,U_00006,84,Other,134.558011,153.102287,NaN,Yes,Yes,Heavy,2021-01-01 05:00:00,...,Normal,1502.944749,3.270787,Moderate,Neutral,38.055861,21.384001,23.031124,46.166093,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8182,U_08183,50,Other,32.419115,162.679903,Diabetes,Yes,No,NaN,2021-12-07 22:00:00,...,Abnormal,2597.379009,0.507414,Low,Sad,37.385576,22.784776,20.453853,81.662776,1
8186,U_08187,75,Other,76.312426,188.610040,Hypertension,No,No,NaN,2021-12-08 02:00:00,...,Normal,2150.019580,1.617131,High,Neutral,36.261740,18.007424,39.622975,36.371495,0
8187,U_08188,75,Other,31.342192,176.912590,Diabetes,Yes,No,Heavy,2021-12-08 03:00:00,...,Abnormal,1850.508372,2.960398,Low,Anxious,35.681559,20.770012,72.133524,58.173399,1
8188,U_08189,25,Other,70.052857,194.116152,NaN,No,No,Moderate,2021-12-08 04:00:00,...,Abnormal,2307.514078,2.682981,Low,Anxious,34.486115,33.828232,72.971645,64.452306,1


In [42]:
print(pizza.columns)

Index(['User_ID', 'Age', 'Gender', 'Weight', 'Height', 'Medical_Conditions',
       'Medication', 'Smoker', 'Alcohol_Consumption', 'Timestamp',
       'Day_of_Week', 'Sleep_Duration', 'Deep_Sleep_Duration',
       'REM_Sleep_Duration', 'Wakeups', 'Snoring', 'Heart_Rate',
       'Blood_Oxygen_Level', 'ECG', 'Calories_Intake', 'Water_Intake',
       'Stress_Level', 'Mood', 'Skin_Temperature', 'Body_Fat_Percentage',
       'Muscle_Mass', 'Health_Score', 'Anomaly_Flag'],
      dtype='object')


In [45]:
# Assuming 'pizza' is your DataFrame

# 1. Identify columns with non-numeric data
non_numeric_cols = pizza.select_dtypes(exclude=['number']).columns

# 2. Drop or encode non-numeric columns
# Option 1: Drop non-numeric columns
# X = pizza.drop(non_numeric_cols.union(['User_ID','Stress_Level']), axis=1).values

# Option 2: One-hot encode non-numeric columns (Recommended)
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') # sparse=False for compatibility with StandardScaler
encoded_data = encoder.fit_transform(pizza[non_numeric_cols])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(non_numeric_cols))
numeric_df = pizza.drop(non_numeric_cols.union(['User_ID','Stress_Level']), axis=1)
X = pd.concat([numeric_df, encoded_df], axis=1).values


# 3. Proceed with scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=True, with_std=True)
X_s = scaler.fit_transform(X)

In [43]:
X = pizza.drop(['User_ID','Stress_Level'], axis=1).values
n, p = X.shape
print(f'n = {n}, p = {p}')

n = 10000, p = 26


In [38]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=True, with_std=True)
X_s = scaler.fit_transform(X)

ValueError: could not convert string to float: 'Other'